In [1]:
from osgeo import gdal
import rasterio  
import os
import rasterio
import numpy as np



In [3]:
# Đường dẫn tới thư mục chứa các file JP2
img_data_path = "/Users/lengochuynh/Downloads/S2A_20240302T045947.SAFE/GRANULE/L1C_T48PYS_A045403_20240302T032031/IMG_DATA"

# Đọc các băng RGB (B04, B03, B02)
bands = {}
for band_name, file_name in zip(["B04", "B03", "B02"], ["T48PYS_20240302T030631_B04.jp2", "T48PYS_20240302T030631_B03.jp2", "T48PYS_20240302T030631_B02.jp2"]):
    file_path = os.path.join(img_data_path, file_name)
    with rasterio.open(file_path) as src:
        bands[band_name] = src.read(1)

# Kết hợp các băng RGB
rgb_image = np.stack([bands["B04"], bands["B03"], bands["B02"]], axis=-1)


In [5]:
import xml.etree.ElementTree as ET

In [7]:
# Đọc file metadata XML
metadata_path = "/Users/lengochuynh/Downloads/S2A_20240302T045947.SAFE/MTD_MSIL1C.xml"
tree = ET.parse(metadata_path)
root = tree.getroot()

In [9]:
# Lấy góc tới mặt trời (Solar Zenith Angle)
solar_angle = float(root.find(".//Mean_Sun_Angle/ZENITH_ANGLE").text)

In [13]:
# In ra toàn bộ cấu trúc XML để tìm đúng thẻ
for elem in root.iter():
    print(elem.tag, elem.text)

In [15]:
solar_angle = 30.0 

In [17]:
# Lấy hệ số chuyển đổi M từ metadata
scale_factors = {
    "B04": float(root.find(".//QUANTIFICATION_VALUE").text),
    "B03": float(root.find(".//QUANTIFICATION_VALUE").text),
    "B02": float(root.find(".//QUANTIFICATION_VALUE").text),
}


In [19]:
# Chuyển đổi từng băng
toa_bands = {}
for band_name in bands:
    toa_bands[band_name] = (bands[band_name] * scale_factors[band_name]) / np.cos(np.radians(solar_angle))


In [21]:
#from Py6S import *

In [23]:
# Khởi tạo đối tượng SixS
#s = SixS()

# Thiết lập các tham số mô phỏng khí quyển
# Ví dụ: sử dụng profile khí quyển nhiệt đới và profile aerosol hàng hải
#s.atmos_profile = AtmosProfile.PredefinedType(AtmosProfile.Tropical)
#s.aero_profile = AeroProfile.PredefinedType(AeroProfile.Maritime)

# Chọn giá trị phản xạ mặt đất, giả sử là đất với phản xạ đồng nhất (0.2)
#s.ground_reflectance = GroundReflectance.HomogeneousLambertian(0.2)
# Thiết lập các tham số geometry (góc của mặt trời và máy quét)
#s.geometry.solar_z = 30  # Góc tới mặt trời
#s.geometry.view_z = 0  # Góc của máy quét (trực tiếp từ trên xuống)
#s.geometry.month = 12  # Tháng thu thập dữ liệu
#s.geometry.day = 7  # Ngày thu thập dữ liệu
# Chạy mô phỏng
#s.run()


In [25]:
surface_reflectance = {}
for band_name, band_data in toa_bands.items():
    dark_pixel_value = np.percentile(band_data, 1)  # Giá trị pixel tối nhất (1%)
    surface_reflectance[band_name] = band_data - dark_pixel_value

In [27]:
# Kết hợp các băng đã hiệu chỉnh
rgb_image = np.stack([surface_reflectance["B04"], surface_reflectance["B03"], surface_reflectance["B02"]], axis=-1)


In [29]:
# Chuẩn hóa để hiển thị
def normalize(array):
    return (array - array.min()) / (array.max() - array.min())

rgb_image = normalize(rgb_image)

In [31]:
import rasterio
from rasterio.enums import Resampling

In [47]:
# Định nghĩa đường dẫn xuất ảnh
output_path = "/Users/lengochuynh/Downloads/tt/rbg.tif"
# Tạo một file raster mới và ghi ảnh đã chuẩn hóa vào đó
with rasterio.open(output_path, 'w', driver='GTiff',
                   height=rgb_image.shape[0], width=rgb_image.shape[1],
                   count=3, dtype=rgb_image.dtype,
                   crs="EPSG:3405") as dst: # Đặt CRS nếu có (VD: EPSG:4326 là WGS84)
                   #transform=dataset.transform) as dst:  # Dùng transform từ dataset (nếu có)
    # Ghi các băng ảnh vào
    for i in range(3):  # 3 băng cho RGB
        dst.write(rgb_image[:, :, i], i + 1)

print(f"Ảnh đã được xuất thành công tại {output_path}")


Ảnh đã được xuất thành công tại /Users/lengochuynh/Downloads/tt/rbg.tif


In [ ]:
metadata_path = "/Users/lengochuynh/Downloads/S2A_20240302T045947.SAFE/MTD_MSIL1C.xml"
tree = ET.parse(metadata_path)
root = tree.getroot()
# Lấy góc tới mặt trời (Solar Zenith Angle)
solar_angle = float(root.find(".//Mean_Sun_Angle/ZENITH_ANGLE").text)
# Lấy hệ số chuyển đổi M từ metadata
scale_factors = {
    "B04": float(root.find(".//QUANTIFICATION_VALUE").text),
    "B03": float(root.find(".//QUANTIFICATION_VALUE").text),
    "B02": float(root.find(".//QUANTIFICATION_VALUE").text),
}
# Chuyển đổi từng băng
toa_bands = {}
for band_name in bands:
    toa_bands[band_name] = (bands[band_name] * scale_factors[band_name]) / np.cos(np.radians(solar_angle))
